<a href="https://colab.research.google.com/github/ljpetkovic/Charcot_KeyBERT_Keyphrase-Vectorizers/blob/main/scripts/KeyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction des mots/phrases-clés avec `keybert` et `keyphrase-vectorizers`

# ✅ `keybert`
* librairie Python pour extraire des mots/phrases-clés les plus similaires à un document en exploitant les plongements BERT<br>
⚠️ on doit spécifier la longueur des n-grammes à extraire, alors que l'on ne sait pas quelle est la longueur optimale<br>
⚠️ la grammaticalité des phrases n'est pas prise en compte

<p align="right"><a href="https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db">Schopf, 2022</a></p>


## 1<sup>e</sup> méthode

In [ ]:
# !pip install keybert[all]

import torch # print(torch.__version__)
import os
from google.colab import drive
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
kw_model = KeyBERT(model=sentence_model)

# Monter le Google Drive
drive.mount('/content/drive')

# Definir les chemins vers les fichiers d'entrée et de sortie
path = '/content/drive/MyDrive/JE_ObTIC_Circulations/data/'
file_name = 'autres_keybert_mmr.txt'
file_path = 'corpus_autres.txt'

# Extraction des mots-clés
with open(os.path.join(path, file_path), 'r') as myfile, open(os.path.join(path, file_name), 'w') as outfile:
    raw_data = myfile.readlines()
    start = 0
    end = 10000 # diminuer le nb de lignes à traiter si la mémoire RAM est épuisée (p. ex. 300 fonctionne bien)
    while len(raw_data) >= end:
        data = " ".join(raw_data[start:end])
        start = end
        end += 10000
        keywords = kw_model.extract_keywords(data, keyphrase_ngram_range=(1, 3), stop_words=None, use_mmr=True, diversity=0.7)
        for k in keywords:
            print(k)
            # Writing each keyword to the output file
            outfile.write(str(k) + '\n')

## 2<sup>e</sup> méthode

In [ ]:
# Installer KeyBERT et toutes les dépendances
!pip install keybert[all]

import torch # print(torch.__version__)
import os
from google.colab import drive
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

# Monter le Google Drive
drive.mount('/content/drive')

# Definir les chemins vers les fichiers d'entrée et de sortie
path = '/content/drive/MyDrive/JE_ObTIC_Circulations/data/'
file_name = 'charcot_keybert_mmr.txt'
file_path = 'corpus_charcot.txt'

# Initialiser le modèle de phrase et le modèle KeyBERT
sentence_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
kw_model = KeyBERT(model=sentence_model)

with open(os.path.join(path, file_name), 'w') as f:
    pass  # Pour vider le fichier


# Définir une fonction pour traiter des morceaux de texte
written_keywords = set()  # Garder une trace des mots-clés qui ont été écrits dans le fichier

def process_text_chunk(text_chunk):
    global written_keywords
    keywords = kw_model.extract_keywords(text_chunk, keyphrase_ngram_range=(1, 3), stop_words=None, use_mmr=True, diversity=0.7)
    with open(os.path.join(path, file_name), 'a+') as liste:
        for keyword, score in keywords:
            if keyword not in written_keywords:
                print(keyword)
                liste.write(keyword + '\n')
                written_keywords.add(keyword)



# Traitez le fichier en morceaux pour éviter de charger l'intégralité du fichier en mémoire
chunk_size = 300  # Définir le nombre de lignes à lire à la fois


try:
    with open(os.path.join(path, file_path), 'r') as file:
      lines_buffer = []
      for line in file:
          lines_buffer.append(line.strip())
          if len(lines_buffer) >= chunk_size:
              data = " ".join(lines_buffer)
              process_text_chunk(data)
              lines_buffer = []  # Réinitialiser le tampon après le traitement

      # Traiter tout le texte restant
      if lines_buffer:
          data = " ".join(lines_buffer)
          process_text_chunk(data)

except Exception as e:
    print(f"An error occurred: {e}")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.6 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39199 sha256=930c05c7ab3fa5285338558e2d140852c901526bbb0c6385610a43b1f88b2aa2
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert
Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
une décision administrative
1838 et de
sont parfaitement susceptibles
recours au médecins
est la loi
183s ne
été assez éclairé
mais ici vous
loi 111 tetelin
magistrature chacun restera
rapporteur mais non
liberté savez vous
organisés autorité administrative
médecins adjoints pourront
aliénation moniale toute
stage de acte
accordé 20 minutes
l3ourneoille un volume
nous devons regretter
objet de véritables
pages avec 61
protestant enfin contre
sa femme et
autorités une enquête
communiquée 323 vons
cet élément fondamental
épidémies ne
un autre travail
trois types principaux
constitution des esprits
rechercher croyons nous
suicide était héréditaire
préexister acte nous
trente ans et
chacun ap porte
délire cette étude
persécuté tout mélancolique
leur apparition ce
paris 1887 334
une très grande
autorités administrative et
les centres corticaux
homme elle disait
336 pathologie mentale
pourra jamais reproduire
tard elle espère
maria copié un


# ✅ `keyphrase-vectorizers`
* librairie Python pour extraire des mots-clés les plus similaires à un document en exploitant les plongements BERT<br>
❇️ pas besoin de spécifier la longueur des n-grammes à extraire, car la librairie l'infère elle-même<br>
❇️ la grammaticalité des phrases est prise en compte grâce aux extractions des parties du discours (p. ex. `<N.*>*<ADJ.*>*<ADJ.*>+`--> _sclérose latérale amyotrophique_)

([Schopf, 2022](https://towardsdatascience.com/enhancing-keybert-keyword-extraction-results-with-keyphrasevectorizers-3796fa93f4db))

In [ ]:
# Pré-requis :
# 1. GPU (sinon, CPU)
#    - si mémoire RAM épuisée, penser à vider la mémoire cache
# 2. excellente connexion de réseau

# Installations
!pip install keyphrase-vectorizers
!pip install deplacy spacy-transformers
!python -m spacy download fr_core_news_lg

# Imports
import spacy
from google.colab import drive
from keyphrase_vectorizers import KeyphraseCountVectorizer
import os

# Charger le modèle français spaCy_lg
nlp = spacy.load("fr_core_news_lg")

# Monter le Google Drive
drive.mount('/content/drive')

# Initialiser le vectoriseur avec le modèle spécifié
vectorizer = KeyphraseCountVectorizer(spacy_pipeline=nlp, pos_pattern='<N.*>*<ADJ.*>*<ADJ.*>+', stop_words='french')

# Définir les chemins vers les fichiers d'entrée et de sortie
path = '/content/drive/MyDrive/JE_ObTIC_Circulations/data/'
input_file_name = 'corpus_charcot.txt' # 'corpus_autres.txt' pour le corpus Autres
output_file_name = 'charcot_keyphrase-vectorizers.txt' # ou 'autres_keyphrase-vectorizers.txt' pour le corpus Autres

# Définir la fonction qui traite les blocs de texte (segmentation)
def process_chunk(chunk):
    # Join the lines into a single string
    data = " ".join(chunk)
    # Fit the vectorizer on the chunk
    vectorizer.fit([data])

# Lire le fichier et le traiter en blocs des 5,000 lignes pour éviter de charger l'intégralité du fichier en mémoire
# Dans le cas des blocs plus grands, la mémoire RAM s'épuise et la session Colab plante
chunk_size = 5000
current_chunk = []

with open(os.path.join(path, input_file_name), 'r') as myfile:
    for line in myfile:
        current_chunk.append(line.strip())
        if len(current_chunk) == chunk_size:
            process_chunk(current_chunk)
            current_chunk = []  # Reset the chunk

    # Traiter toutes les lignes restantes dans le dernier bloc
    if current_chunk:
        process_chunk(current_chunk)

# Après avoir traité tous les blocs, extraire et sauvegarder les mots-clés dans un fichier
keyphrases = vectorizer.get_feature_names_out()

with open(os.path.join(path, output_file_name), 'w') as output_file:
    for keyphrase in keyphrases:
        print(keyphrase)
        output_file.write(keyphrase + '\n')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
cause immédiate
- mentale
lésions inflammatoires aiguës
fibrineuse
contractilité faradique
slowy fatal
œil consécutifs
nerf s
abdominales
lésions inflammatoires
existence permanente
cellules plates isolées
lew ré-partition inégale
1a partie supérieure
capables
rouge sombre
\ sens spéciaux
larges
ralysie générale
suppression totale
insidieux
celle
supérieurs
insister'.jamais
paralysie hystérique
opposés
souples
gers
secousse clonique
nir
_ oeuvres complètes
régions centrales
apparence fibroïde
arcades dentaires
translucides
genre particulier
tige rigide
accidents hystériques
sion irritative
rouge vif
rameaux cutanés
injection hypodermique
élevé
origine hystérique
affections gangreneuses extérieures
concomitants
âge sénile^
règle absolue
rôle prédominant
tissu conjoncttf fibrillaire
rhumatisme noueux
incen
attitudes bizarres
éruptions cutanées spéciales
étendue variable
jambe prédomine
stase sanguine
tretende
p

# PatternRank
* `keybert` + `keyphrase-vectorizers` = PatternRank
 * extraction des phrases-clés les plus similaires à un document
 * préservation de leur grammaticalité grâce aux motifs POS
* _cf._ [Schopf _et al._, 2022](https://arxiv.org/pdf/2210.05245.pdf)





In [7]:
# !pip install keyphrase-vectorizers
# !pip install keybert
# !pip install flair
from keyphrase_vectorizers import KeyphraseCountVectorizer
from keybert import KeyBERT
from flair.embeddings import TransformerDocumentEmbeddings
import os
from google.colab import drive
# Monter le Google Drive
drive.mount('/content/drive')

# Ajuster les chemins
path = '/content/drive/MyDrive/ObTIC/ateliers/extraction_mots_cles/corpus/'
input_file_name = 'test.txt'
output_file_name = '../output/test_output.txt'

# Initialiser le modèle KeyBERT multilingue
kw_model = KeyBERT(model=TransformerDocumentEmbeddings('google-bert/bert-base-multilingual-cased'))

# Ajuster les paramètres
vectorizer = KeyphraseCountVectorizer(spacy_pipeline='fr_core_news_lg', pos_pattern='<N.*>+<ADJ.*>*', stop_words='french')

with open(os.path.join(path, input_file_name), 'r') as input_file, \
     open(os.path.join(path, output_file_name), 'w') as output_file:
    raw_data = input_file.readlines()
    start = 0
    end = 22 # diviser le texte en tranches
    while start < len(raw_data):  # s'assurer que l'on traite toutes les données
        data = " ".join(raw_data[start:end]).replace('\n', ' ')  # Joindre les lignes and gérer les nouvelles lignes
        start = end
        end += 22
        try:
            # extraire les phrases-clés
            kp = kw_model.extract_keywords(data, vectorizer=vectorizer)
            for k in kp:
                print(k)
                output_file.write(str(k) + '\n')
        except ValueError as e:
            print(f"An error occurred while processing chunks starting at line {start}: {e}")
            # Accessoirement, écrire un message ou gérer l'erreur si besoin
            # output_file.write("Pas de phrases-clés extraites pour cette tranche.\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
('14 planches', 0.8301)
('progrès médical', 0.787)
('leçons', 0.7721)
('oeuvres complètes', 0.7623)
('35 figures', 0.7263)
('œuvres complè', 0.9246)
('réimpression', 0.9131)
('pachyméningite spinale', 0.9117)
('preuve convaincante', 0.8918)
('édition précédente', 0.89)
